In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%matplotlib inline

In [ ]:
# CPU:
!pip install torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# GPU:
!pip install torch==1.7.0+cu101 torchvision==0.8.1+cu101 torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchaudio
import numpy as np



import matplotlib.pyplot as plt
import IPython.display as ipd
from tqdm.notebook import tqdm

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 159.3MB 100kB/s 
     |████████████████████████████████| 11.8MB 31.6MB/s 
     |████████████████████████████████| 7.6MB 3.7MB/s 
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.0+cpu which is incompatible.
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101
  Found existing installation: torchvision 0.9.1+cu101
    Uninstalling torchvision-0.9.1+cu101:
      Successfully uninstalled torchvision-0.9.1+cu101
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 735.3MB 25kB/s 
     |████████████████████████████████| 12.7MB 227kB/s 
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.0+cu101 which is incompatible.
  Found existing installation: torch 1.7.0+cpu
    Uninstalling torch-1.7.0+c

/usr/local/lib/python3.7/dist-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


In [ ]:
!wget -O - https://kheafield.com/code/kenlm.tar.gz | tar xz
%mkdir kenlm/build
%cd kenlm/build
!ls
!cmake ..
!make -j 4

--2021-05-14 09:12:55--  https://kheafield.com/code/kenlm.tar.gz
Resolving kheafield.com (kheafield.com)... 35.196.63.85
Connecting to kheafield.com (kheafield.com)|35.196.63.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 491090 (480K) [application/x-gzip]
Saving to: ‘STDOUT’

-                   100%[===================>] 479.58K   523KB/s    in 0.9s    

2021-05-14 09:12:56 (523 KB/s) - written to stdout [491090/491090]

/content/kenlm/build
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- 

In [ ]:
%cd /content

/content


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
def _levenshtein_distance(ref, hyp):

    m = len(ref)
    n = len(hyp)

    # special case
    if ref == hyp:
        return 0
    if m == 0:
        return n
    if n == 0:
        return m

    if m < n:
        ref, hyp = hyp, ref
        m, n = n, m

    # use O(min(m, n)) space
    distance = np.zeros((2, n + 1), dtype=np.int32)

    # initialize distance matrix
    for j in range(0,n + 1):
        distance[0][j] = j

    # calculate levenshtein distance
    for i in range(1, m + 1):
        prev_row_idx = (i - 1) % 2
        cur_row_idx = i % 2
        distance[cur_row_idx][0] = i
        for j in range(1, n + 1):
            if ref[i - 1] == hyp[j - 1]:
                distance[cur_row_idx][j] = distance[prev_row_idx][j - 1]
            else:
                s_num = distance[prev_row_idx][j - 1] + 1
                i_num = distance[cur_row_idx][j - 1] + 1
                d_num = distance[prev_row_idx][j] + 1
                distance[cur_row_idx][j] = min(s_num, i_num, d_num)

    return distance[m % 2][n]



def char_errors(reference, hypothesis, ignore_case=False, remove_space=False):

    if ignore_case == True:
        reference = reference.lower()
        hypothesis = hypothesis.lower()

    join_char = ' '
    if remove_space == True:
        join_char = ''

    reference = join_char.join(filter(None, reference.split(' ')))
    hypothesis = join_char.join(filter(None, hypothesis.split(' ')))

    edit_distance = _levenshtein_distance(reference, hypothesis)
    return float(edit_distance), len(reference)


def cer(reference, hypothesis, ignore_case=False, remove_space=False):

    edit_distance, ref_len = char_errors(reference, hypothesis, ignore_case,
                                         remove_space)

    if ref_len == 0:
        raise ValueError("Length of reference should be greater than 0.")

    cer = float(edit_distance) / ref_len
    return cer

def my_wer(reference, hypothesis):
    if reference == hypothesis:
      return 0
    else:
      return 1


In [ ]:
class TextTransform:
  def __init__(self):
    char_map_str = """
        a 1
        b 2
        c 3
        d 4
        e 5
        f 6
        g 7
        h 8
        i 9
        j 10
        k 11
        l 12
        m 13
        n 14
        o 15
        p 16
        q 17
        r 18
        s 19
        t 20
        u 21
        v 22
        w 23
        x 24
        y 25
        z 26
        """
    self.char_map = {}
    self.index_map = {}
    for line in char_map_str.strip().split('\n'):
      ch, index = line.split()
      self.char_map[ch] = int(index)
      self.index_map[int(index)] = ch
    
  def text_to_int(self, text):
    int_sequence = []
    for c in text:
      ch = self.char_map[c]
      int_sequence.append(ch)
    return int_sequence

  def int_to_text(self, labels):
    string = []
    for i in labels:
      string.append(self.index_map[i])
    return ''.join(string)
  
text_transform = TextTransform()

In [ ]:
!git clone --recursive https://github.com/parlance/ctcdecode.git
!cd ctcdecode && pip install .

Cloning into 'ctcdecode'...
remote: Enumerating objects: 1063, done.
remote: Total 1063 (delta 0), reused 0 (delta 0), pack-reused 1063
Receiving objects: 100% (1063/1063), 759.71 KiB | 3.65 MiB/s, done.
Resolving deltas: 100% (513/513), done.
Submodule 'third_party/ThreadPool' (https://github.com/progschj/ThreadPool.git) registered for path 'third_party/ThreadPool'
Submodule 'third_party/kenlm' (https://github.com/kpu/kenlm.git) registered for path 'third_party/kenlm'
Cloning into '/content/ctcdecode/third_party/ThreadPool'...
remote: Enumerating objects: 82, done.        
remote: Total 82 (delta 0), reused 0 (delta 0), pack-reused 82        
Cloning into '/content/ctcdecode/third_party/kenlm'...
remote: Enumerating objects: 13808, done.        
remote: Counting objects: 100% (121/121), done.        
remote: Compressing objects: 100% (71/71), done.        
remote: Total 13808 (delta 65), reused 81 (delta 37), pack-reused 13687        
Receiving objects: 100% (13808/13808), 5.49 MiB | 

In [ ]:
!wget https://www.cse.iitb.ac.in/~pjyothi/cs753/train_list.txt
!wget https://www.cse.iitb.ac.in/~pjyothi/cs753/test_list.txt

--2021-05-14 09:18:41--  https://www.cse.iitb.ac.in/~pjyothi/cs753/train_list.txt
Resolving www.cse.iitb.ac.in (www.cse.iitb.ac.in)... 103.21.127.134
Connecting to www.cse.iitb.ac.in (www.cse.iitb.ac.in)|103.21.127.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 70437 (69K) [text/plain]
Saving to: ‘train_list.txt’

train_list.txt      100%[===================>]  68.79K  --.-KB/s    in 0.1s    

2021-05-14 09:18:42 (701 KB/s) - ‘train_list.txt’ saved [70437/70437]

--2021-05-14 09:18:42--  https://www.cse.iitb.ac.in/~pjyothi/cs753/test_list.txt
Resolving www.cse.iitb.ac.in (www.cse.iitb.ac.in)... 103.21.127.134
Connecting to www.cse.iitb.ac.in (www.cse.iitb.ac.in)|103.21.127.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19957 (19K) [text/plain]
Saving to: ‘test_list.txt’

test_list.txt       100%[===================>]  19.49K  --.-KB/s    in 0s      

2021-05-14 09:18:43 (208 MB/s) - ‘test_list.txt’ saved [19957/19957]



In [ ]:
from torchaudio.datasets import SPEECHCOMMANDS
import os
from pathlib import Path

TRAIN_FILE = 'train_list.txt'
TEST_FILE  = 'test_list.txt'

train_indices = []
test_indices = []

with open(TRAIN_FILE) as f:
  train_indices = [int(line.strip()) for line in f]

with open(TEST_FILE) as f:
  test_indices = [int(line.strip()) for line in f]

class SubsetSC(SPEECHCOMMANDS):
  def __init__(self, subset: str = None):
    path_of_dataset = "./"
    super().__init__(path_of_dataset, download=True)
    walker = sorted(str(p) for p in Path(self._path).glob('*/*.wav'))
    self._path = path_of_dataset + 'SpeechCommands/speech_commands_v0.02'
    
    def load_list(filename):
      print(self._path)
      filepath = os.path.join(self._path, filename)
      with open(filepath) as fileobj:
        return [os.path.join(self._path, line.strip()) for counter, line in enumerate(fileobj)]
  
    if subset == "validation":
      self._walker = load_list("validation_list.txt")
    elif subset == "testing":
      self._walker = load_list("testing_list.txt")
      self._walker = [self._walker[i] for i in test_indices]
    elif subset == "training":
      excludes = load_list("validation_list.txt") + load_list("testing_list.txt")
      excludes = set(excludes)
      self._walker = [w for w in self._walker if w not in excludes]
      self._walker = [self._walker[i] for i in train_indices]

In [ ]:
train_set = SubsetSC("training")
test_set = SubsetSC("testing")


./SpeechCommands/speech_commands_v0.02
./SpeechCommands/speech_commands_v0.02
./SpeechCommands/speech_commands_v0.02


In [ ]:
print(len(train_set))
print(len(test_set))

waveform, sample_rate, label, speaker_id, utterance_number = train_set[0]

print(train_set[0])

12000
4000
(tensor([[9.1553e-05, 2.4414e-04, 2.4414e-04,  ..., 3.6621e-04, 6.4087e-04,
         7.9346e-04]]), 16000, 'backward', '017c4098', 2)


In [ ]:
labels = set()
for i,datapoint in enumerate(train_set):
  if i % 100 == 0:
    print(i)
  labels.add(datapoint[2])

labels = sorted(list(labels))
labels

CORPUS = 'corpus.txt'
with open(CORPUS,'w') as f:
  for label in labels:
    f.write(label + '\n')

!./kenlm/build/bin/lmplz --discount_fallback -o 2 < corpus.txt > lm.arpa

In [ ]:
waveform_last, *_ = train_set[-1]
ipd.Audio(waveform_last.numpy(), rate=sample_rate)

In [ ]:
def GreedyDecoder(output, blank_label=0, collapse_repeated=True):
  arg_maxes = torch.argmax(output, dim=2)
  decodes = []
  for i, args in enumerate(arg_maxes):
    decode = []
    for j, index in enumerate(args):
      if index != blank_label:
        if collapse_repeated and j != 0 and index == args[j-1]:
          continue
        decode.append(index.item())
    decodes.append(text_transform.int_to_text(decode))
  return decodes

In [ ]:
class CNNLayerNorm(nn.Module):
    """Layer normalization built for cnns input"""
    def __init__(self, n_feats):
        super(CNNLayerNorm, self).__init__()
        self.layer_norm = nn.LayerNorm(n_feats)

    def forward(self, x):
        # x (batch, channel, feature, time)
        x = x.transpose(2, 3).contiguous() # (batch, channel, time, feature)
        x = self.layer_norm(x)
        return x.transpose(2, 3).contiguous() # (batch, channel, feature, time) 


class ResidualCNN(nn.Module):
    """Residual CNN inspired by https://arxiv.org/pdf/1603.05027.pdf
        except with layer norm instead of batch norm
    """
    def __init__(self, in_channels, out_channels, kernel, stride, dropout, n_feats):
        super(ResidualCNN, self).__init__()

        self.cnn1 = nn.Conv2d(in_channels, out_channels, kernel, stride, padding=kernel//2)
        self.cnn2 = nn.Conv2d(out_channels, out_channels, kernel, stride, padding=kernel//2)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.layer_norm1 = CNNLayerNorm(n_feats)
        self.layer_norm2 = CNNLayerNorm(n_feats)

    def forward(self, x):
        residual = x  # (batch, channel, feature, time)
        x = self.layer_norm1(x)
        x = F.gelu(x)
        x = self.dropout1(x)
        x = self.cnn1(x)
        x = self.layer_norm2(x)
        x = F.gelu(x)
        x = self.dropout2(x)
        x = self.cnn2(x)
        x += residual
        return x # (batch, channel, feature, time)


class BidirectionalGRU(nn.Module):

    def __init__(self, rnn_dim, hidden_size, dropout, batch_first):
        super(BidirectionalGRU, self).__init__()

        self.BiGRU = nn.GRU(
            input_size=rnn_dim, hidden_size=hidden_size,
            num_layers=1, batch_first=batch_first, bidirectional=True)
        self.layer_norm = nn.LayerNorm(rnn_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.layer_norm(x)
        x = F.gelu(x)
        x, _ = self.BiGRU(x)
        x = self.dropout(x)
        return x

class SpeechRecognitionModel(nn.Module):
  def __init__(self):
    super(SpeechRecognitionModel, self).__init__()

    n_feats = 64
    rnn_dim = 256
    self.cnn = nn.Conv2d(1, 32, 3, stride=2, padding=3//2)  # cnn for extracting heirachal features

        # n residual cnn layers with filter size of 32
    self.rescnn_layers = nn.Sequential(*[
            ResidualCNN(32, 32, kernel=3, stride=1, dropout=0.1, n_feats=n_feats) 
            for _ in range(3)
        ])
    self.fully_connected = nn.Linear(n_feats*32, rnn_dim)
    self.birnn_layers = nn.Sequential(*[
            BidirectionalGRU(rnn_dim=rnn_dim if i==0 else rnn_dim*2,
                             hidden_size=rnn_dim, dropout=0.1, batch_first=i==0)
            for i in range(5)
        ])

    self.classifier = nn.Sequential(
        nn.Linear(512, 256),
        nn.GELU(),
        nn.Dropout(0.1),
        nn.Linear(256,27)
    )

  def forward(self, x):
    x = self.cnn(x)
    x = self.rescnn_layers(x)
    sizes = x.size()
    x = x.view(sizes[0], sizes[1] * sizes[2], sizes[3])  # (batch, feature, time)
    x = x.transpose(1, 2) # (batch, time, feature)
    x = self.fully_connected(x)
    x = self.birnn_layers(x)
    x = self.classifier(x)
    return x


In [ ]:
model = SpeechRecognitionModel()

In [ ]:
from ctcdecode import CTCBeamDecoder

characters = list("_abcdefghijklmnopqrstuvwxyz") 

decoder = CTCBeamDecoder(
    characters,
  )

smart_decoder = CTCBeamDecoder(
    characters,
    model_path = 'lm.arpa'    
)

In [ ]:
path_to_model = 'drive/MyDrive/CS753/model_state_dict.pt'
model.load_state_dict(torch.load(path_to_model))
print(model)

SpeechRecognitionModel(
  (cnn): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (rescnn_layers): Sequential(
    (0): ResidualCNN(
      (cnn1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (cnn2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
      (layer_norm1): CNNLayerNorm(
        (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      )
      (layer_norm2): CNNLayerNorm(
        (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      )
    )
    (1): ResidualCNN(
      (cnn1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (cnn2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
      (layer_norm1): CNNLayerNorm(
        (layer_norm): LayerNorm((64,),

**Adversarial Attack**

In [ ]:
def attack(example, target_label):

  iterations = 1000
  learning_rate = 1
  l2penalty=float('inf') 
  tau =  2000

  mel_transform = torchaudio.transforms.MelSpectrogram()

  orig_waveform, sample_rate, label, speaker_id, utterance_number = example


  orig_waveform = orig_waveform
  orig_waveform.requires_grad = True
  delta = torch.tensor([0.]*(example[0].size()[1]), requires_grad=True)
  
  rescale = torch.tensor([1.],requires_grad = True)

  optimizer = optim.AdamW([delta], learning_rate)
  criterion = nn.CTCLoss(blank=0)

  best_delta = None
  for i in range(iterations):
    apply_delta = torch.clamp(delta, -tau, tau)*rescale


    new_input = apply_delta + orig_waveform

    noise = torch.normal(0,0.001, size=new_input.size())

    pass_in = torch.clamp(new_input , -2**15, 2**15-1)

    spec = mel_transform(pass_in).squeeze(0).transpose(0,1)

    spectogram = nn.utils.rnn.pad_sequence([spec], batch_first=True).unsqueeze(1).transpose(2,3)
  
    target = text_transform.text_to_int(target_label.lower())
    target = target + [0]*(len(label)-len(target))
    target = torch.Tensor(target)
    target = nn.utils.rnn.pad_sequence([target], batch_first=True)

    input_length = [spec.shape[0]//2]

    target_length = [len(label)]
    
    optimizer.zero_grad()

    output = model(spectogram)
    output = F.log_softmax(output, dim=2)
    output = output.transpose(0,1)

    decode_output = F.softmax(output, dim=2)

    beam_results, beam_scores, timesteps, out_lens = decoder.decode(decode_output)
    beam_results = beam_results.tolist()

    decoded_preds = [text_transform.int_to_text(beam_results[i][0][:out_lens[i][0]]) for i in range(1)]
    
    orig_waveform.retain_grad()
    spec.retain_grad()
    spectogram.retain_grad()

    ctcloss = criterion(output, target, input_length,target_length)      

    if not np.isinf(l2penalty):
        loss = tf.reduce_mean((self.new_input-self.original)**2,axis=1) + l2penalty*ctcloss
    else:
        loss = ctcloss 

    model.eval()
    with torch.no_grad():
      my_output = model(spectogram)

      decode_output = F.softmax(my_output, dim=2)

      beam_results, beam_scores, timesteps, out_lens = decoder.decode(decode_output)
      beam_results = beam_results.tolist()

      decoded_preds = [text_transform.int_to_text(beam_results[i][0][:out_lens[i][0]]) for i in range(1)]

    model.train()

    #if i % 50 == 0:
      #print(loss)
      #print(decoded_preds)
    
    if decoded_preds[0] == target_label:
      #print("Temporary Solution Found.")
      if rescale*tau > torch.max(torch.abs(delta)).item():
        rescale = torch.max(torch.abs(delta)).item()/tau
      rescale *= 0.9

      best_delta = apply_delta.clone().detach()
    
    
    loss.backward()
    optimizer.step()

  return best_delta


In [ ]:
model.eval()
with torch.no_grad():
  new = example[0] + deltaa
  mel_transform = torchaudio.transforms.MelSpectrogram()
  spect = mel_transform(new).squeeze(0).transpose(0,1)

  spectogram = nn.utils.rnn.pad_sequence([spect], batch_first=True).unsqueeze(1).transpose(2,3)

  output = model(spectogram)
  # output = F.log_softmax(output, dim=2)
  # output = output.transpose(0,1)

  decode_output = F.softmax(output, dim=2)
        
  beam_results, beam_scores, timesteps, out_lens = decoder.decode(decode_output)
  beam_results = beam_results.tolist()

  # beam_results_lm, beam_scores_lm, timesteps_lm, out_lens_lm = smart_decoder.decode(decode_output)
  # beam_results_lm = beam_results_lm.tolist()
        
  # decoded_preds = GreedyDecoder(output.transpose(0,1))
        
  decoded_preds = [text_transform.int_to_text(beam_results[0][0][:out_lens[0][0]])]
  # decoded_preds_lm = [text_transform.int_to_text(beam_results_lm[0][0][:out_lens_lm[0][0]])]

  print(decoded_preds)
# print(decoded_preds_lm)

['hello']


/usr/local/lib/python3.7/dist-packages/torchaudio/functional.py:318: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  "At least one mel filterbank has all zero values. "


In [ ]:
def get_SNR(orig,adversarial):
  delta = adversarial - orig
  #orig = orig/torch.sum(orig)
  #delta = delta/torch.sum(orig)

  signal_energy_dB = (10*torch.log(torch.sum(torch.square(orig)))).detach().numpy()
  noise_energy_dB = (10*torch.log(torch.sum(torch.square(delta)))).detach().numpy()
  SNR_dB = (10*torch.log(torch.sum(torch.square(orig))/torch.sum(torch.square(delta)))).detach().numpy()

  return np.squeeze(signal_energy_dB)/np.log(10),np.squeeze(noise_energy_dB)/np.log(10),np.squeeze(SNR_dB)/np.log(10)

In [ ]:
def model_decode(waveform):
  mel_transform = torchaudio.transforms.MelSpectrogram()
  model.eval()
  spect = mel_transform(waveform).squeeze(0).transpose(0,1)

  spectogram = nn.utils.rnn.pad_sequence([spect], batch_first=True).unsqueeze(1).transpose(2,3)

  output = model(spectogram)
  # output = F.log_softmax(output, dim=2)
  # output = output.transpose(0,1)

  decode_output = F.softmax(output, dim=2)
  

  beam_results, beam_scores, timesteps, out_lens = decoder.decode(decode_output)
  beam_results = beam_results.tolist()

  decoded_preds = [text_transform.int_to_text(beam_results[i][0][:out_lens[i][0]]) for i in range(1)]


  # decoded_preds = GreedyDecoder(output.transpose(0,1))

  return decoded_preds

In [ ]:
def evaluate_adv_example(example,target_label):

  orig_waveform, sample_rate, orig_label, speaker_id, utterance_number = example

  deltaa = attack(example, target_label)

  adv_waveform = (orig_waveform + deltaa).detach()
  print("Original Audio")
  display(ipd.Audio((orig_waveform).detach().numpy(), rate=sample_rate))
  print("label of original waveform: ",orig_label)
  pred_label = model_decode(orig_waveform)
  print("Predicted label of original waveform: ",pred_label)
  print("\n")
  print("Adversarial Audio")
  display(ipd.Audio((adv_waveform).detach().numpy(), rate=sample_rate))
  print("Target label : ",target_label)
  adv_pred_label = model_decode(adv_waveform)
  print("Predicted label of adversarial waveform : ",adv_pred_label)
  signal_energy_dB,noise_energy_dB,SNR_dB = get_SNR(orig_waveform, adv_waveform)
  print("Signal Energy: ",signal_energy_dB," dB")
  print("Noise Energy: ",noise_energy_dB," dB")
  print("SNR:           ",SNR_dB," dB")

  print("#######################################################################")

**Evaluation of Adversarial examples**

In [ ]:
import random

target_labels = ['hello','protect','speech','predict','viva','lazy','crazy','igloo','software', 'happy', 'left', 'learn', 'marvin' , 'visual']

for target in target_labels:
  orig_label = ''
  while len(orig_label) < len(target):
    id = random.randint(0,11999)
    example = train_set[id]
    orig_waveform, sample_rate, orig_label, speaker_id, utterance_number = example
  print(orig_label)
  evaluate_adv_example(example,target)

sheila


/usr/local/lib/python3.7/dist-packages/torchaudio/functional.py:318: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  "At least one mel filterbank has all zero values. "


Original Audio


label of original waveform:  sheila
Predicted label of original waveform:  ['sheila']


Adversarial Audio


Target label :  hello
Predicted label of adversarial waveform :  ['hello']
Signal Energy:  19.162743689985263  dB
Noise Energy:  2.183691000739621  dB
SNR:            16.979053103421133  dB
#######################################################################
forward
Original Audio


...

Target label :  learn
Predicted label of adversarial waveform :  ['learn']
Signal Energy:  19.95893477758948  dB
Noise Energy:  -15.826934507323202  dB
SNR:            35.78586928491268  dB
#######################################################################
forward
Original Audio


label of original waveform:  forward
Predicted label of original waveform:  ['forward']


Adversarial Audio


Target label :  marvin
Predicted label of adversarial waveform :  ['marvin']
Signal Energy:  14.115892710029811  dB
Noise Energy:  -28.161673792579514  dB
SNR:            42.27756981601327  dB
#######################################################################
sheila
Original Audio


label of original waveform:  sheila
Predicted label of original waveform:  ['sheila']


Adversarial Audio


Target label :  visual
Predicted label of adversarial waveform :  ['visual']
Signal Energy:  22.438013669164853  dB
Noise Energy:  -28.19352885810852  dB
SNR:            50.63154584067732  dB
#######################################################################
